In [16]:
import cv2
import tensorflow
from PIL import ImageFont, ImageDraw, Image
import numpy as np


In [60]:



def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def transform_image(img,ang_range,shear_range,trans_range,brightness=0):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over.

    A Random uniform distribution is used to generate different parameters for transformation

    '''
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2

    # Brightness


    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)

    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))

    if brightness == 1:
      img = augment_brightness_camera_images(img)

    return img

In [78]:
img = cv2.imread("img.jpg")

img = cv2.resize(img, (50, 50))



imgAr = np.zeros((50, 50, 3), dtype=np.uint8)
imgAr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
pil_image = Image.fromarray(imgAr)

# Draw non-ascii text onto img
font = ImageFont.truetype("./NotoSansEgyptianHieroglyph.ttf", 43 )
draw = ImageDraw.Draw(pil_image)



unicodeChar = "𓀀"

draw.text((2, -10), unicodeChar, font=font, fill=(0,0,0,255))

# Convert back to Numpy array and switch back from RGB to BGR
img = np.asarray(pil_image)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


for i in range(10):
    img1= img
    img1 = transform_image(img1, 10, 3, 3, 0)


    cv2.imshow("img", img1)
    cv2.waitKey(0)
    
    # It is for removing/deleting created GUI window from screen
    # and memory
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)